# Fine-tuning

Dentro de los ejemplos presentados anteriormente se utilizo a Lora para ajustar los modelos de OpenChat y GPT2, pero, para que hacer un ajuste fino de un modelo LLM, o mejor dicho, ¿cuando es necesario hacer un ajuste fino de un modelo LLM? Para endender el concepto de ajuste fino se debe tener presente el hecho que s necesitan gran cantidad de datos para entrenarlo y ademas, capacidades de computo para entrenar el modelo. Por lo tanto, si se desea hacer un ajuste fino de un modelo LLM, primero se deben tener en cuenta los siguientes puntos:

- Limtaciones del modelo para la tarea que se desea realizar.
- Uso de prompting para mejorar el rendimiento del modelo.
- Si RAG (Generación Aumentada por Recuperación) es una opción viable para la tarea que se desea realizar

Se deben tener en cuenta todos los puntos anteriores antes de decidir hacer un ajuste fino de un modelo de LLM, ya que, para realizar este ajuste se necesita utilizar gran cantidad de recursos computacionales que en la mayoria de los casos no estan disponibles. En recomendacio particular, si se desea usar un LLM, para una tarea especifica, se recomienda usar un modelo que ya este ajustado, agotar medologias de prompting y RAG, y si aun asi no se obtiene el resultado deseado, entonces se debe considerar hacer un ajuste fino del modelo. A continuacion haremos un ejemplo incorrecto de ajuste fino de un modelo LLM, para que se entienda el concepto y se pueda ver como se realiza este proceso y como las respuestas del modelo son completamente incorrectas, ya que el modelo no esta entrenado para la tarea que se desea realizar.

In [13]:
from datasets import load_dataset

dataset = load_dataset("daily_dialog", split="train")
dataset

Dataset({
    features: ['dialog', 'act', 'emotion'],
    num_rows: 11118
})

In [14]:
dialogues = []

for dialog in dataset['dialog']:
    for i in range(len(dialog) - 1):
        user_utt = dialog[i]
        bot_utt = dialog[i + 1]
        sample = f"<|user|> {user_utt} <|bot|> {bot_utt}"
        dialogues.append(sample)


In [15]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 no tiene pad_token por defecto

# Tokenizamos todos los pares
tokenized_inputs = tokenizer(dialogues, truncation=True, padding="max_length", max_length=128, return_tensors="pt")


In [16]:
from torch.utils.data import Dataset

class ConversationalDataset(Dataset):
    def __init__(self, tokenized_data):
        self.input_ids = tokenized_data["input_ids"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "labels": self.input_ids[idx]
        }

train_dataset = ConversationalDataset(tokenized_inputs)

In [17]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-chatbot",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True  # Si estás en Colab con GPU
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [18]:
trainer.train()

Step,Training Loss
100,2.868900
200,0.775800
300,0.766200
400,0.747200
500,0.730400
600,0.731300
700,0.737000
800,0.732400
900,0.717000
1000,0.728500


TrainOutput(global_step=7131, training_loss=0.6972235790761419, metrics={'train_runtime': 1537.4333, 'train_samples_per_second': 148.401, 'train_steps_per_second': 4.638, 'total_flos': 1.4903836213248e+16, 'train_loss': 0.6972235790761419, 'epoch': 3.0})

In [21]:
model.save_pretrained("./gpt2-chatbot")
tokenizer.save_pretrained("./gpt2-chatbot")

('./gpt2-chatbot/tokenizer_config.json',
 './gpt2-chatbot/special_tokens_map.json',
 './gpt2-chatbot/vocab.json',
 './gpt2-chatbot/merges.txt',
 './gpt2-chatbot/added_tokens.json')

In [33]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def chat(prompt):
    input_text = f"<|user|> {prompt} <|bot|>"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    output = model.generate(
        input_ids,
        max_length=150,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)
    return response.strip()

# Ejemplo
print(chat("¿Is Paris the capital of France?"))


Yes , it is .


In [34]:
chat_history = []

def chat_with_context(user_input, chat_history, max_turns=6):
    # Agregar turno del usuario
    chat_history.append(f"<|user|> {user_input}")

    # Limitar el historial si es muy largo
    if len(chat_history) > max_turns * 2:
        chat_history = chat_history[-(max_turns * 2):]

    # Construir prompt con historial
    prompt = "\n".join(chat_history) + "\n<|bot|>"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generar respuesta
    output = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + 100,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        do_sample=True
    )

    # Extraer solo la respuesta nueva
    response = tokenizer.decode(output[0][input_ids.shape[1]:], skip_special_tokens=True).strip()

    # Agregar turno del bot al historial
    chat_history.append(f"<|bot|> {response}")

    return response, chat_history


In [35]:
while True:
    user_input = input("Tú: ")
    if user_input.lower() in ["salir", "exit", "quit"]:
        break
    response, chat_history = chat_with_context(user_input, chat_history)
    print("Bot:", response)


Bot: It's very nice .
Bot: Yes , it is . It's very nice .
Bot: Yes .


KeyboardInterrupt: Interrupted by user